In [1]:
import pandas as pd
#PREPARO DATOS PARA FUNCION 1 Y 2
df_games=pd.read_csv("games.csv")


In [2]:
import numpy as np

# Elimina filas con valores no finitos
df_games = df_games.dropna(subset=['id'])
df = df_games[~df_games['id'].isin([np.inf, -np.inf])]
# Convertir la columna "id" a tipo de dato int porque esta florante para q no ocupe espacio
df_games['id'] = df_games['id'].astype(int)

In [3]:
df_item=pd.read_csv("df_items1.csv")
df_item.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,10,Counter-Strike,14,0,sprayandpray168543483
2,10,Counter-Strike,12,0,BlazingTheAmazing
3,10,Counter-Strike,10,0,WolfOfCSGOL
4,10,Counter-Strike,259,0,InstigatorAU


In [4]:
df_games=df_games.drop(['publisher','app_name', 'reviews_url','specs','price', 'early_access','developer'],axis=1)

In [5]:
df_item=df_item.drop(['item_name','playtime_2weeks'],axis=1)
df_item = df_item.set_index('item_id')


In [6]:
# Tengo los DataFrames df_generos y df_item
# Realiza el merge de ambos DataFrames
merged_df = pd.merge(df_games, df_item, left_on='id', right_on='item_id', how='inner')
# Extraer los primeros cuatro caracteres de "release_date" y convertirlos a enteros
merged_df=merged_df.dropna()
merged_df['anio'] = merged_df['release_date'].str[:4]
# extraigo el año

In [7]:
#borro los años 0
merged_df=merged_df[merged_df['anio']!="0000"]

In [ ]:
#para ver si la fecha tiene formato correcto despues de todas las transformaciones
merged_df['release_date'] = pd.to_datetime(merged_df['release_date'], format='%Y-%m-%d', errors='coerce')

In [ ]:
# Agrupar por "genres" y anio sumar la columna "time_forever"
#resultados_agrupados = merged_df.groupby('genres','anio')['playtime_forever']

resultados_agrupados = merged_df.groupby(['genres', 'anio'])['playtime_forever'].sum()

# Convierte el índice en una columna
resultados_agrupados = resultados_agrupados.reset_index()



In [ ]:
# GRABO GENEROS FECHA FUNCION 1
resultados_agrupados.to_parquet('Generos_fecha.parquet',index=False)

In [8]:
#Para la funcion 2 USUARIOS TIEMPO 
# Agrupar por  y sumar la columna "time_forever"
resultados_agrupados2 = merged_df.groupby(['genres','anio','user_id'])['playtime_forever'].sum()
# Convierte el índice en una columna
resultados_agrupados2 = resultados_agrupados2.reset_index()

In [11]:
# VOY A ELIMINAR LA MITAD DEL DF PARA VER SI FUNCIONA RENDER
# Crear una serie booleana para seleccionar filas con índices impares
filas_a_eliminar = resultados_agrupados2.index % 2 == 1

# Utilizar la función drop para eliminar las filas con índices impares
resultados_agrupados2 = resultados_agrupados2[~filas_a_eliminar]

In [12]:
resultados_agrupados2


,genres,anio,user_id,playtime_forever
0,Action,1983,2Ta4,18
2,Action,1983,76561197966936422,331
4,Action,1983,76561197969020980,98
6,Action,1983,76561197971951483,27
8,Action,1983,76561197973470219,0
...,...,...,...,...
3435532,Web Publishing,2017,76561198045638432,1877
3435534,Web Publishing,2017,76561198068809841,886
3435536,Web Publishing,2017,DougyFed,329
3435538,Web Publishing,2017,N47H4NI3L,1635


In [10]:
# Creo parquet para la segunda funcion
# Escribe el DataFrame en un archivo Parquet

resultados_agrupados2.to_parquet('Usuarios_tiempo2.parquet', index=False)


In [ ]:
import pandas as pd
# FUNCIONES 3 Y 4 PREPARO LOS ARVHICOS
df_games=pd.read_csv("games.csv")

df_games.dropna(inplace=True)
df_ur=pd.read_csv('user_reviews.csv')
df_item=pd.read_csv('df_items1.csv')

In [ ]:
df_item=df_item[['item_id','item_name']]

In [ ]:
# valores unicos para despues hacer el merge
df_item = df_item[['item_id', 'item_name']].drop_duplicates()
df_item

In [ ]:
# CREO CSV PARA FUNCIONES 3-4 Y 5
df_ur=pd.read_csv("funcion3.csv")
df_ur['item_id']=df_ur['item_id'].astype(int)
df_item=pd.read_csv("df_items1.csv")
df_item.dropna(inplace=True)
df_item=df_item.drop(['playtime_forever','playtime_2weeks','user_id'],axis=1)
# Realizo el merge de ambos DataFrames
merged_df = pd.merge(df_ur, df_item, left_on='item_id', right_on='item_id', how='inner')
merged_df=merged_df.sort_values(by='item_id')

In [ ]:
# Escribo csv
# Guarda el DataFrame en formato Parquet sin incluir el índice
merged_df.to_parquet('func_3.parquet', index=False)

#merged_df.to_csv('func_3.csv', index=False) 